In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

Here are some possible business questions:

- What factors contribute most heavily to the price? 
    - Make a model to predict the price - one that's interpretable and perhaps one (like a neural network) that's not interpretable but does a better job of price prediction.
    - Which listings are underpriced - meaning that the host could make more money by increasing the price, or travelers can get a good deal by booking them?
    - Perhaps extend beyond predicting only base prices to predicting prices at different times of year.
- What should those listings with poor overall review scores do in order to get better reviews in the future?
- What can we discern about the character of each neighborhood using the descriptions?

In [2]:
listings = pd.read_csv("listings.csv")
calendar = pd.read_csv("calendar.csv")

Definitions for many (though not all) of the fields can be found [here](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit#gid=982310896).

Drop columns that have all `NA` values.

In [3]:
for df in [listings, calendar]:
    df.dropna(axis=1, how='all', inplace=True)

Drop rows in `listings` that have `NA` in the `price` field, and do the same for the `price` and `adjusted_price` fields in `calendar`.

In [4]:
listings.dropna(subset=['price'], inplace=True)
calendar.dropna(subset=['price', 'adjusted_price'], inplace=True)

Observe that a dollar sign preceeds the number in several columns; I will remove this so that the field is treated as numeric.

In [5]:
listings['price'] = listings.apply((lambda x: x['price'].replace('$','').replace(',','')), axis=1)
calendar['price'] = calendar.apply((lambda x: x['price'].replace('$','').replace(',','')), axis=1)
calendar['adjusted_price'] = calendar.apply((lambda x: x['adjusted_price'].replace('$','').replace(',','')), axis=1)

In [6]:
listings['price'] = listings['price'].astype(np.float64)
calendar['price'] = calendar['price'].astype(np.float64)
calendar['adjusted_price'] = calendar['adjusted_price'].astype(np.float64)

Similarly, some of the fields are expressed as percentages; I will remove the percent signs and change the entries to floats.

In [7]:
listings['host_response_rate'] = listings.apply(lambda x: float(x['host_response_rate'].replace('%','')) if pd.notnull(x['host_response_rate']) else x['host_response_rate'], axis=1)
listings['host_acceptance_rate'] = listings.apply(lambda x: float(x['host_acceptance_rate'].replace('%','')) if pd.notnull(x['host_acceptance_rate']) else x['host_acceptance_rate'], axis=1)

# 1. Price prediction

## Data preparation

Split into `X` and `y`.

In [8]:
X = listings.drop('price', axis=1)
y = listings['price']

I also need to deal with dates appropriately. The relevant columns are `host_since`, `first_review`, and `last_review`. I could scrap these columns altogether, but it might be useful to know the length of time since the date for each row. 

In [9]:
for col in ['host_since', 'first_review', 'last_review']:
    X[col] = pd.to_datetime(X['last_scraped']) - pd.to_datetime(X[col])
    X[col] = X.apply(lambda x: x[col].days, axis=1)

Certain columns in `X` are not useful for prediction (for example, URLs), and should be removed.

In [10]:
to_drop = ['id', 'listing_url', 'scrape_id', 'last_scraped', 'name']
to_drop += ['description', 'neighborhood_overview', 'picture_url', 'host_id']
to_drop += ['host_url', 'host_name', 'host_about', 'host_thumbnail_url']
to_drop += ['host_picture_url', 'license', 'calendar_last_scraped']
X.drop(labels=to_drop, axis=1, inplace=True)

Certain columns (`host_verifications` and `amenities`) are lists of options; I'd like to split these into different columns.

The first step is to change these entries into actual Python lists instead of strings (annoyingly, the two relevant columns are formatted slightly differently, with the roles of single and double quotes being interchanged between them).

In [11]:
X['host_verifications'] = X.apply(lambda x: x['host_verifications'].replace('[','').replace(']','').replace('\'','').split(', '), axis=1)

In [12]:
X['amenities'] = X.apply(lambda x: x['amenities'].replace('\"','').replace('[','').replace(']','').split(', '), axis=1)

What are the possible values in the `host_verifications` lists?

In [13]:
distinct_host_verifications = set()
def find_distinct_hvs(x):
    for hv in x['host_verifications']:
        distinct_host_verifications.add(hv)
X.apply(lambda x: find_distinct_hvs(x), axis=1);

In [14]:
len(distinct_host_verifications)

20

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer

In [16]:
m = MultiLabelBinarizer()
hv_df = pd.DataFrame(m.fit_transform(X['host_verifications']), columns=m.classes_, index=X.index)

In [17]:
X = pd.concat([X, hv_df], axis=1)

In [18]:
X.rename(columns={'None' : 'HV_None'}, inplace=True)
# I don't want a column to be called just 'None', so I changed it a bit

Now I can drop the original `host_verifications` column.

In [19]:
X.drop('host_verifications', axis=1, inplace=True)

Now I'd like to do the same for the `amenities` column. What are the possible values in those lists?

In [20]:
distinct_amenities = set()
def find_distinct_amenities(x):
    for a in x['amenities']:
        distinct_amenities.add(a)
X.apply(lambda x: find_distinct_amenities(x), axis=1)
len(distinct_amenities)

990

Ok, that's a lot of amenities. I don't really want to vastly blow up the number of features like this, so I'll just take the 20 most popular amenities.

In [21]:
from collections import defaultdict
from operator import itemgetter

In [22]:
amenity_counts = defaultdict(int) # Initializes to zero for each value
def update_counts(x):
    for a in x['amenities']:
        amenity_counts[a] += 1
X.apply(lambda x: update_counts(x), axis=1);

In [23]:
top_20 = sorted(amenity_counts.items(), key=itemgetter(1), reverse=True)[:20]

In [24]:
top_20[:5]

[('Smoke alarm', 6359),
 ('Essentials', 6185),
 ('Wifi', 6141),
 ('Kitchen', 5930),
 ('Carbon monoxide alarm', 5926)]

In [25]:
top_20_amenities = dict(top_20).keys()

Now I want to get rid of the other amenities in each row.

In [26]:
def find_amenities(x):
    amenities = []
    for a in top_20_amenities:
        if a in x['amenities']:
            amenities.append(a)
    return amenities

In [27]:
X['amenities'] = X.apply(lambda x: find_amenities(x), axis=1)

Now I can create the dummy variables and drop the original `amenities` column.

In [28]:
m = MultiLabelBinarizer()
amenities_df = pd.DataFrame(m.fit_transform(X['amenities']), columns=m.classes_, index=X.index)
X = pd.concat([X, amenities_df], axis=1)
X.drop('amenities', axis=1, inplace=True)

Next, I'll get dummies for the rest of the categorical variables.

In [29]:
X = pd.get_dummies(X, drop_first=True)

In [30]:
X.shape

(6528, 740)

Now I need to deal with `NA` values for the numeric columns. There's no totally ideal way to do this, but to move forward I'll impute using the mean.

In [31]:
fill_mean = lambda col: col.fillna(col.mean())
X = X.apply(fill_mean)

## Modeling and Evaluation

Split into train and test sets.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

Instantiate the model, including normalization in the pipeline:

In [33]:
pipe = make_pipeline(StandardScaler(), LinearRegression())

Fit the model to the training data:

In [34]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [35]:
pipe.score(X_train, y_train)

0.42794538848965413

In [36]:
pipe.score(X_test, y_test)

-2.358149984883228e+26

In [37]:
pipe.predict(X_train)

array([  744.64194006,  2353.47440005,    69.95444006, ...,
         978.64194006,   318.64194006, -2290.04555994])

In [38]:
y_train

2018    149.0
6068    126.0
3740    180.0
4104    124.0
6286    158.0
        ...  
3772    131.0
5191    131.0
5226    300.0
5390    102.0
860     221.0
Name: price, Length: 5222, dtype: float64

In [39]:
pipe.predict(X_test)

array([4105.47440005,  700.64194006,  869.95444006, ...,  650.64194006,
       -845.35805994,  190.64194006])

Wow, this is really terrible. I think I need to aggressively get rid of features to correct this - I have way too many irrelevant features.

In [40]:
def test_cutoffs(X, y, cutoffs):
    for cutoff in cutoffs:
        pipe = make_pipeline(StandardScaler(), LinearRegression())
        X_cutoff = X.iloc[:, np.where((X.sum()/len(X) > cutoff) == True)[0]]
        X_train, X_test, y_train, y_test = train_test_split(
            X_cutoff, y, test_size=0.3, random_state=42)
        pipe.fit(X_train, y_train)
        train_score = pipe.score(X_train, y_train)
        test_score = pipe.score(X_test, y_test)
        print(f"Cutoff = {cutoff: .3g}, train_score = {train_score: .3g}, test_score = {test_score: .3g}, n_features = {X_train.shape[1]}")
    #return results

In [41]:
test_cutoffs(X, y, [1000, 1, .99, .9, .7, .5, .3, .2, .1, .05, .01, 0.005, 0])

Cutoff =  1e+03, train_score =  0.00221, test_score =  0.00202, n_features = 4
Cutoff =  1, train_score =  0.0703, test_score =  0.0569, n_features = 37
Cutoff =  0.99, train_score =  0.0705, test_score =  0.0575, n_features = 39
Cutoff =  0.9, train_score =  0.205, test_score =  0.168, n_features = 46
Cutoff =  0.7, train_score =  0.292, test_score =  0.226, n_features = 61
Cutoff =  0.5, train_score =  0.297, test_score =  0.231, n_features = 69
Cutoff =  0.3, train_score =  0.299, test_score =  0.234, n_features = 75
Cutoff =  0.2, train_score =  0.298, test_score =  0.23, n_features = 78
Cutoff =  0.1, train_score =  0.317, test_score =  0.248, n_features = 85
Cutoff =  0.05, train_score =  0.329, test_score =  0.255, n_features = 98
Cutoff =  0.01, train_score =  0.363, test_score =  0.267, n_features = 164
Cutoff =  0.005, train_score =  0.378, test_score =  0.266, n_features = 202
Cutoff =  0, train_score =  0.433, test_score = -1.94e+26, n_features = 739


In [42]:
pipe.named_steps['linearregression'].coef_;

In [43]:
pipe = make_pipeline(StandardScaler(), LinearRegression())
X_cutoff = X.iloc[:, np.where((X.sum()/len(X) > .01) == True)[0]]
X_train, X_test, y_train, y_test = train_test_split(
    X_cutoff, y, test_size=0.3, random_state=42)
pipe.fit(X_train, y_train)
score = pipe.score(X_test, y_test)
y_pred = pipe.predict(X_test)

Horrific.

What do we expect to be useful for price prediction? My guesses: Its size (number of people, number of bedrooms and bathrooms), the neighborhood, and the review scores.

The relevant columns are `neighbourhood_cleansed`, `accommodates`, `bathrooms_text`, `bedrooms`, and `beds`. I'll hold off on including review scores for now.

In [45]:
X_small = listings[['neighbourhood_cleansed','accommodates','bathrooms_text','bedrooms','beds']]

In [46]:
X_small = pd.get_dummies(X_small, drop_first=True)

In [47]:
y_small = y.copy()

In [72]:
X.loc[6524]['beds']

5.0

In [73]:
y.loc[6524]

264.0

In [48]:
X_small

,accommodates,bedrooms,beds,neighbourhood_cleansed_Archer Heights,neighbourhood_cleansed_Armour Square,neighbourhood_cleansed_Ashburn,neighbourhood_cleansed_Auburn Gresham,neighbourhood_cleansed_Austin,neighbourhood_cleansed_Avalon Park,neighbourhood_cleansed_Avondale,...,bathrooms_text_5 baths,bathrooms_text_5.5 baths,bathrooms_text_6 baths,bathrooms_text_6.5 baths,bathrooms_text_7 baths,bathrooms_text_7.5 baths,bathrooms_text_8 shared baths,bathrooms_text_Half-bath,bathrooms_text_Private half-bath,bathrooms_text_Shared half-bath
0,1,1.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,2.0,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,1.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,1.0,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6523,4,2.0,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6524,8,3.0,5.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6525,8,3.0,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6526,6,2.0,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
X_small['bedrooms'] = X_small.apply(lambda x: x['beds'] if not(x['bedrooms']>0) else x['bedrooms'], axis=1)
X_small['beds'] = X_small.apply(lambda x: x['bedrooms'] if not(x['beds']>0) else x['beds'], axis=1)

In [50]:
sum(X_small.isna()['beds'])

6

In [51]:
X_small.loc[X_small.isna()['bedrooms']]

,accommodates,bedrooms,beds,neighbourhood_cleansed_Archer Heights,neighbourhood_cleansed_Armour Square,neighbourhood_cleansed_Ashburn,neighbourhood_cleansed_Auburn Gresham,neighbourhood_cleansed_Austin,neighbourhood_cleansed_Avalon Park,neighbourhood_cleansed_Avondale,...,bathrooms_text_5 baths,bathrooms_text_5.5 baths,bathrooms_text_6 baths,bathrooms_text_6.5 baths,bathrooms_text_7 baths,bathrooms_text_7.5 baths,bathrooms_text_8 shared baths,bathrooms_text_Half-bath,bathrooms_text_Private half-bath,bathrooms_text_Shared half-bath
4074,2,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4327,0,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5590,2,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6015,4,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6416,2,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6516,4,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
to_drop = X_small.loc[X_small['accommodates'] == 0].index
X_small.drop(to_drop, inplace=True)
y_small.drop(to_drop, inplace=True)

In [53]:
X_small['bedrooms'] = X_small.apply(lambda x: x['accommodates'] if not(x['bedrooms'] > 0) else x['bedrooms'], axis=1)
X_small['beds'] = X_small.apply(lambda x: x['accommodates'] if not(x['beds'] > 0) else x['beds'], axis=1)

In [54]:
sum(X_small.isna()['bedrooms'])

0

In [55]:
X_small_train, X_small_test, y_small_train, y_small_test = train_test_split(
    X_small, y_small, test_size=.3, random_state=42)
pipe = make_pipeline(StandardScaler(), LinearRegression())
pipe.fit(X_small_train, y_small_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [56]:
pipe.score(X_small_train, y_small_train)

0.033706592134797875

In [57]:
test_cutoffs(X_small_train, y_small_train, [1, .99, .9, .7, .5, .3, .2, .1, .05, .01, 0])

Cutoff =  1, train_score =  0.00104, test_score =  0.00114, n_features = 3
Cutoff =  0.99, train_score =  0.00104, test_score =  0.00114, n_features = 3
Cutoff =  0.9, train_score =  0.00104, test_score =  0.00114, n_features = 3
Cutoff =  0.7, train_score =  0.00104, test_score =  0.00114, n_features = 3
Cutoff =  0.5, train_score =  0.00339, test_score =  0.00249, n_features = 4
Cutoff =  0.3, train_score =  0.00339, test_score =  0.00249, n_features = 4
Cutoff =  0.2, train_score =  0.00339, test_score =  0.00249, n_features = 4
Cutoff =  0.1, train_score =  0.0337, test_score =  0.0133, n_features = 8
Cutoff =  0.05, train_score =  0.0339, test_score =  0.0141, n_features = 13
Cutoff =  0.01, train_score =  0.0347, test_score =  0.0135, n_features = 38
Cutoff =  0, train_score =  0.0375, test_score = -5.76e+20, n_features = 104


In [58]:
pipe = make_pipeline(StandardScaler(), LinearRegression())
X_small_cutoff = X_small.iloc[:, np.where((X_small.sum()/len(X_small) > .05) == True)[0]]
X_small_train, X_small_test, y_small_train, y_small_test = train_test_split(
    X_small_cutoff, y_small, test_size=0.3, random_state=42)
pipe.fit(X_small_train, y_small_train)
score = pipe.score(X_small_test, y_small_test)
y_small_pred = pipe.predict(X_small_test)

In [63]:
u = ((y_test - y_pred)**2).sum()
v = ((y_test - y_test.mean())**2).sum()
print(1 - u/v)

0.2666255577618506


The coefficient of determination, $R^2$, is at least positive; that is, the predictors do better than one that just always predicts the mean price. Still, there is a long way to go to actually get good predictions.

In [64]:
from sklearn.linear_model import ElasticNet

In [79]:
def test_cutoffs_Elastic(X, y, cutoffs):
    for cutoff in cutoffs:
        pipe = make_pipeline(StandardScaler(), ElasticNet())
        X_cutoff = X.iloc[:, np.where((X.sum()/len(X) > cutoff) == True)[0]]
        X_train, X_test, y_train, y_test = train_test_split(
            X_cutoff, y, test_size=0.3, random_state=42)
        pipe.fit(X_train, y_train)
        train_score = pipe.score(X_train, y_train)
        test_score = pipe.score(X_test, y_test)
        print(f"Cutoff = {cutoff: .3g}, train_score = {train_score: .3g}, test_score = {test_score: .3g}, n_features = {X_train.shape[1]}")
    #return results

In [86]:
def fit_model(X, y, model, cutoff):
    pipe = make_pipeline(StandardScaler(), model)
    X_cutoff = X.iloc[:, np.where((X.sum()/len(X) > cutoff) == True)[0]]
    X_train, X_test, y_train, y_test = train_test_split(
        X_cutoff, y, test_size=0.3, random_state=42)
    pipe.fit(X_train, y_train)
    return pipe, X_train, X_test, y_train, y_test

Note `ElasticNet` uses 5-fold cross-validation for hyperparameter tuning by default (I don't need to set up the CV myself).

In [122]:
test_cutoffs_Elastic(X, y, [1000, 1, .99, .9, .7, .5, .3, .2, .1, .05, .01, 0.005, 0])

Cutoff =  1e+03, train_score =  0.00193, test_score =  0.00188, n_features = 4
Cutoff =  1, train_score =  0.0403, test_score =  0.0432, n_features = 37
Cutoff =  0.99, train_score =  0.0404, test_score =  0.0434, n_features = 39
Cutoff =  0.9, train_score =  0.17, test_score =  0.158, n_features = 46
Cutoff =  0.7, train_score =  0.241, test_score =  0.221, n_features = 61
Cutoff =  0.5, train_score =  0.246, test_score =  0.226, n_features = 69
Cutoff =  0.3, train_score =  0.249, test_score =  0.228, n_features = 75
Cutoff =  0.2, train_score =  0.249, test_score =  0.228, n_features = 78
Cutoff =  0.1, train_score =  0.265, test_score =  0.243, n_features = 85
Cutoff =  0.05, train_score =  0.278, test_score =  0.251, n_features = 98
Cutoff =  0.01, train_score =  0.293, test_score =  0.263, n_features = 164
Cutoff =  0.005, train_score =  0.298, test_score =  0.265, n_features = 202
Cutoff =  0, train_score =  0.322, test_score =  0.272, n_features = 739


In [87]:
pipe, X_train, X_test, y_train, y_test = fit_model(X, y, ElasticNet(), 0)

In [88]:
pipe.score(X_test, y_test)

0.27212546707272434

Not much better than my best linear model above, which has $R^2 = 0.267$.

What are the most important features?

In [127]:
coeffs = pipe.named_steps['elasticnet'].coef_
df = pd.DataFrame(np.array([X_train.columns, coeffs, np.abs(coeffs)]).T, columns=['feature','coeff', 'abs_coeff'])
df = df.sort_values(by='abs_coeff', ascending=False)
df.head(30)

,feature,coeff,abs_coeff
75,Smoke alarm,-458.492116,458.492116
656,neighbourhood_cleansed_West Town,286.968933,286.968933
63,Essentials,-253.750141,253.750141
354,host_location_US,227.243397,227.243397
67,Heating,-224.736842,224.736842
72,Microwave,164.297672,164.297672
62,Dishes and silverware,-163.453631,163.453631
73,Refrigerator,156.602889,156.602889
25,last_review,147.420884,147.420884
65,Hair dryer,140.898815,140.898815


In [125]:
len(df.loc[df['abs_coeff']==0])

104

Interesting. I can't say I've learned too much from this... I would say that some amenities are correlated with higher prices, though others are negatively correlated and I'm not sure why that is. For example, `Microwave` is positively correlated while `Smoke alarm` is negatively correlated. A strategy a host could use to try to make a higher price work would be to add the amenities for which the coefficients are positive, such as a microwave, a hair dryer, and a refrigerator.

I'd like to build some better models, even if they are less interpretable. Things to try:

- k-nearest-neighbors
- Random forest
- Neural network